## Import packages

In [1]:
"""
CBottle Video Conditional Animation
====================================

Create conditional weather forecast animations using CBottleVideo with ERA5 data.
"""

import torch
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.colors
import numpy as np
import pandas as pd
import os
import gc
from datetime import datetime, timedelta
import cartopy.crs as ccrs

from earth2studio.data import WB2ERA5
from earth2studio.lexicon import WB2Lexicon, CBottleLexicon
from earth2studio.models.dx import CBottleInfill
from earth2studio.models.px import CBottleVideo
from earth2studio.data.utils import fetch_data

/usr2/postdoc/jlin404/.conda/envs/cbottle_env_experimental/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuration
os.makedirs("outputs", exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Parameters - CONFIGURE THESE
VIDEO_VARIABLE = "t850"
N_FRAMES = 12  # 0-66 hours in 6-hour steps
SEED = 42
INTERPOLATE = True

date_dict = {'year': 2022, 'month': 6, 'day': 1, 'hour': 0, 'minute': 0, 'second': 0}

def generate_datetime_array(year, month, day, hour=0, minute=0, second=0):
    """
    Generate a numpy array of 12 datetime objects spaced 6 hours apart.
    
    Parameters:
    -----------
    year : int
        Year
    month : int
        Month (1-12)
    day : int
        Day of month
    hour : int, optional
        Hour (0-23), default 0
    minute : int, optional
        Minute (0-59), default 0
    second : int, optional
        Second (0-59), default 0
    
    Returns:
    --------
    numpy.ndarray
        Array of 12 datetime64[ns] objects spaced 6 hours apart
    """
    # Create the starting datetime
    start_time = datetime(year, month, day, hour, minute, second)
    
    # Generate 12 timestamps spaced 6 hours apart
    times = [start_time + timedelta(hours=6*i) for i in range(12)]
    
    # Convert to numpy array with datetime64[ns] dtype
    times_array = np.array(times, dtype="datetime64[ns]")
    
    return times_array

# Example usage:
times = generate_datetime_array(date_dict['year'], date_dict['month'], date_dict['day'])
print(times)
print(f"\nShape: {times.shape}")
print(f"Dtype: {times.dtype}")

Using device: cuda
['2022-06-01T00:00:00.000000000' '2022-06-01T06:00:00.000000000'
 '2022-06-01T12:00:00.000000000' '2022-06-01T18:00:00.000000000'
 '2022-06-02T00:00:00.000000000' '2022-06-02T06:00:00.000000000'
 '2022-06-02T12:00:00.000000000' '2022-06-02T18:00:00.000000000'
 '2022-06-03T00:00:00.000000000' '2022-06-03T06:00:00.000000000'
 '2022-06-03T12:00:00.000000000' '2022-06-03T18:00:00.000000000']

Shape: (12,)
Dtype: datetime64[ns]


In [3]:
# ============================================================================
# Step 1: Determine Available ERA5 Variables
# ============================================================================
print("\nDetermining available ERA5 variables...")
wb2_vars = set(WB2Lexicon.VOCAB.keys())
cbottle_vars = list(CBottleLexicon.VOCAB.keys())
available_in_era5 = sorted([v for v in cbottle_vars if v in wb2_vars])

print(f"Using {len(available_in_era5)} ERA5 variables for conditioning")


Determining available ERA5 variables...
Using 34 ERA5 variables for conditioning


In [4]:
# ============================================================================
# Step 2: Fetch and Infill ERA5 Data
# ============================================================================
print("\nFetching ERA5 data...")
era5_ds = WB2ERA5()

# Fetch ERA5 data ONCE
era5_x, era5_coords = fetch_data(era5_ds, times, available_in_era5, device=device)
print(f"ERA5 data shape: {era5_x.shape}")

# Load and run CBottleInfill
print("\nRunning CBottleInfill...")
package_infill = CBottleInfill.load_default_package()
cbottle_infill = CBottleInfill.load_model(
    package_infill,
    input_variables=available_in_era5,
    sampler_steps=18
)
cbottle_infill = cbottle_infill.to(device)
cbottle_infill.set_seed(SEED)

# Run infilling
infilled_x, infilled_coords = cbottle_infill(era5_x, era5_coords)
print(f"Infilled data shape: {infilled_x.shape}")

# CRITICAL: Free CBottleInfill and ERA5 data from GPU
del cbottle_infill, era5_x, era5_coords
torch.cuda.empty_cache()
gc.collect()
print("✓ Freed CBottleInfill from GPU memory")


Fetching ERA5 data...


Fetching WB2 data:   0%|          | 0/408 [00:00<?, ?it/s]

2025-11-19 16:59:36.509 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: tcwv at 2022-06-02T06:00:00
2025-11-19 16:59:36.510 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: v50 at 2022-06-02T06:00:00
2025-11-19 16:59:36.511 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: z300 at 2022-06-03T18:00:00
2025-11-19 16:59:36.512 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: tcwv at 2022-06-02T12:00:00
2025-11-19 16:59:36.513 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: v50 at 2022-06-02T12:00:00
2025-11-19 16:59:36.513 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: z50 at 2022-06-01T00:00:00
2025-11-19 16:59:36.514 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: tcwv at 2022-06-02T18:00:00
20

Fetching WB2 data:   0%|          | 0/408 [00:00<?, ?it/s]

2025-11-19 16:59:36.710 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: u500 at 2022-06-01T06:00:00
2025-11-19 16:59:36.711 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: z300 at 2022-06-01T06:00:00
2025-11-19 16:59:36.712 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: t200 at 2022-06-01T12:00:00
2025-11-19 16:59:36.713 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: u500 at 2022-06-01T12:00:00
2025-11-19 16:59:36.713 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: z300 at 2022-06-01T12:00:00
2025-11-19 16:59:36.715 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: t200 at 2022-06-01T18:00:00
2025-11-19 16:59:36.716 | DEBUG    | earth2studio.data.wb2:fetch_array:241 - Fetching WB2 zarr array for variable: u500 at 2022-06-01T18:00:00

Fetching WB2 data: 100%|██████████| 408/408 [00:08<00:00, 49.26it/s]


ERA5 data shape: torch.Size([12, 1, 34, 721, 1440])

Running CBottleInfill...
Infilled data shape: torch.Size([12, 1, 45, 721, 1440])
✓ Freed CBottleInfill from GPU memory


In [5]:
# ============================================================================
# Step 3: Prepare Conditional Input
# ============================================================================
print("\nPreparing conditional input...")

# Add batch dimension if needed
if len(infilled_x.shape) == 5:  # [time, lead_time, variable, lat, lon]
    x_cond = infilled_x.unsqueeze(0)  # [batch, time, lead_time, variable, lat, lon]
else:
    x_cond = infilled_x

print(f"Conditional input shape: {x_cond.shape}")

if INTERPOLATE:
    x_cond_masked = x_cond.clone()
    x_cond_masked[:, 1:-1, :, :, :, :] = float('nan')  # Mask middle timesteps
    x_cond = x_cond_masked




Preparing conditional input...
Conditional input shape: torch.Size([1, 12, 1, 45, 721, 1440])


In [6]:
# ============================================================================
# Step 4: Run CBottleVideo Inference
# ============================================================================
print("\nLoading CBottleVideo...")
package_video = CBottleVideo.load_default_package()
cbottle_video = CBottleVideo.load_model(package_video, seed=SEED)
cbottle_video = cbottle_video.to(device)

# Update coordinates for CBottleVideo
coords_cond = cbottle_video.input_coords()
coords_cond["time"] = times
coords_cond["batch"] = np.array([0])
coords_cond["variable"] = infilled_coords["variable"]

print("Running conditional video generation...")
iterator = cbottle_video.create_iterator(x_cond, coords_cond)

# CRITICAL: Move x_cond off GPU after starting iterator
# (Keep a CPU copy if you need it later)
x_cond_cpu = x_cond.cpu()
del x_cond, infilled_x, infilled_coords
torch.cuda.empty_cache()
print("✓ Moved input data to CPU")

# Collect outputs (moved to CPU immediately)
outputs = []
coords_list = []


Loading CBottleVideo...
Running conditional video generation...
✓ Moved input data to CPU


In [7]:
for step, (output, output_coords) in enumerate(iterator):
    lead_time = output_coords["lead_time"][0]
    hours = int(lead_time / np.timedelta64(1, "h"))
    print(f"  Step {step}: +{hours}h")

    # CRITICAL: Move to CPU immediately
    outputs.append(output.cpu())
    coords_list.append(output_coords)

    # Free GPU memory after each step
    del output
    if step % 3 == 0:  # Periodic cleanup
        torch.cuda.empty_cache()

    if step >= N_FRAMES - 1:
        break

# CRITICAL: Free CBottleVideo from GPU
del cbottle_video
torch.cuda.empty_cache()
gc.collect()
print("✓ Freed CBottleVideo from GPU memory")

OutOfMemoryError: CUDA out of memory. Tried to allocate 6.75 GiB. GPU 0 has a total capacity of 44.39 GiB of which 5.92 GiB is free. Including non-PyTorch memory, this process has 38.47 GiB memory in use. Of the allocated memory 36.11 GiB is allocated by PyTorch, and 1.86 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)